In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
import math
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input director
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [58]:
# Load data khi lấy từ lớp full connected
features_train = np.load("/kaggle/input/ge-features/features_train.npy")

In [59]:
features_train.shape

(8502, 6400)

In [60]:
# # Merge train and valid's label
train_label_df = pd.read_csv("/kaggle/input/ge-muse-split/train_label.csv")
valid_label_df = pd.read_csv("/kaggle/input/ge-muse-split/val_label.csv")
label_df = pd.concat((train_label_df, valid_label_df), axis = 0)
label_df.shape

(8502, 7)

In [61]:
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()

label_pca = label_df["Rhythm"]
label_enc = lb.fit_transform(label_pca)
label_enc

array([0, 1, 1, ..., 2, 1, 2])

In [72]:
y_test = lb.fit_transform(pd.read_csv("/kaggle/input/ge-muse-split/test_label.csv")["Rhythm"].values)
y_test.shape

(2126,)

In [ ]:
X_train = features_train
y_train = label_enc
X_test = np.load("/kaggle/input/ge-features/features_test.npy")

clf = LogisticRegression()
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [ 0.01,0.1,1, 10],
    'solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky'],
    'max_iter': [100,200]
}

grid = GridSearchCV(clf, param_grid, refit = True, verbose = 3) 
  
grid.fit(X_train, y_train) 

In [ ]:
print(grid.best_estimator_)
print(grid.best_params_)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
y_test

In [ ]:
from sklearn.metrics import classification_report

print("Kết quả không áp dụng PCA:")
print(classification_report(y_test, y_pred, target_names= lb.classes_))

result = classification_report(y_test, y_pred, target_names= lb.classes_, output_dict = True)
result_df = pd.DataFrame(result).transpose()
result_df

result_df.to_csv('./result.csv', index=True)